# **PyTorch를 활용한 Trajectory Prediction**

## **Google Drive Mount 하기**

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

## **데이터 처리를 위한 라이브러리 불러오기**

In [2]:
import random

In [3]:
from io import open
from os import path
import pickle

In [4]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
from tqdm import tqdm

## **PyTorch 라이브러리 불러오기**

In [6]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

## **데이터셋 불러오기**

In [ ]:
!unzip "/content/gdrive/MyDrive/LSTMTracking/dataset/trajectory-prediction.zip"

## **데이터셋 살펴보기**

In [8]:
df = pd.read_csv("/content/WholeVdata2.csv")

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head(10)

## **PyTorch DataLoader 클래스 정의**

In [12]:
class TrajectoryDataset(Dataset) :
    def __init__(self, csvPath="/content/WholeVdata2.csv") :
        # Inheritance
        super(TrajectoryDataset, self).__init__()

        # Initialize Variable
        self.csvPath = csvPath

        # store X as a list, each element is a 100*42(len*# attributes) np array [velx; velY; x; y; acc; angle]*7
        # store Y as a list, each element is a 100*4(len*# attributes) np array[velx; velY; x; y]
        self.framesX, self.framesY = [], []

        # Function-Calling
        self.loadData()
        self.normalizeData()

    def __len__(self) :
        return len(self.framesX)

    def __getitem__(self, index) :
        singleData = self.framesX[index]
        singleLabel = self.framesY[index]

        return (singleData, singleLabel)

    def loadData(self) :
        dataS = pd.read_csv(self.csvPath)
        maxVehicleNum = np.max(dataS.Vehicle_ID.unique())
        for vid in dataS.Vehicle_ID.unique() :
            print(f"{vid} and {maxVehicleNum}")
            frameOri = dataS[dataS.Vehicle_ID == vid]
            frame = frameOri[["Local_X", "Local_Y", "v_Acc", "Angle",
                              "L_rX", "L_rY", "L_rAcc", "L_angle",
                              "F_rX", "F_rY", "F_rAcc", "F_angle",
                              "LL_rX", "LL_rY", "LL_rAcc", "LL_angle",
                              "LF_rX", "LF_rY", "LF_rAcc", "LF_angle",
                              "RL_rX", "RL_rY", "RL_rAcc", "RL_angle",
                              "RF_rX", "RF_rY", "RF_rAcc", "RF_angle"]]
            frame = np.asarray(frame)
            frame[np.where(frame > 4000)] = 0 # assign all 5000 to 0

            # remove anomalies, which has a discontinuious local x or local y
            dis = frame[1:,:2] - frame[:-1,:2]
            dis = np.sqrt(np.power(dis[:,0],2)+np.power(dis[:,1],2))

            index = np.where(dis > 10)
            if not (index[0].all) :
                continue

            # smooth the data column wise
            # window size = 5, polynomial order = 3
            frame =  scipy.signal.savgol_filter(frame, window_length=5, polyorder=3, axis=0)

            # calculate velX and velY according to localX and localY for all vehicles
            allVehicles = []

            for i in range(7) :
                velX = (frame[1:,0+i*4]-frame[:-1, 0+i*4])/0.1
                velAvgX = (velX[1:]+velX[:-1])/2.0
                velX1 = [2.0*velX[0]- velAvgX[0]]
                velEndX = [2.0*velX[-1]- velAvgX[-1]];
                velX = np.array(velX1 + velAvgX.tolist() + velEndX)

                velY = (frame[1:,1+i*4]-frame[:-1, 1+i*4])/0.1
                velAvgY = (velY[1:]+velY[:-1])/2.0
                velY1 = [2.0*velY[0]- velAvgY[0]]
                velEndY = [2.0*velY[-1]-velAvgY[-1]]
                velY = np.array(velY1 + velAvgY.tolist() + velEndY)

                if isinstance(allVehicles,(list)) :
                    allVehicles = np.vstack((velX, velY))
                else:
                    allVehicles = np.vstack((allVehicles, velX.reshape(1,-1)))
                    allVehicles = np.vstack((allVehicles, velY.reshape(1,-1)))

            allVehicles = np.transpose(allVehicles)
            totalFrameData = np.concatenate((allVehicles[:,:2], frame), axis=1)

            # split into several frames each frame have a total length of 100, drop sequence smaller than 130
            if totalFrameData.shape[0] < 130 :
                continue

            X = totalFrameData[:-29,:]
            Y = totalFrameData[29:,:4]

            count = 0
            for i in range(X.shape[0]-100) :
                if random.random() > 0.2 :
                    continue

                if count>20:
                    break

                self.framesX = self.framesX + [X[i:i+100,:]]
                self.framesY = self.framesY + [Y[i:i+100,:]]

                count += 1

    def normalizeData(self) :
        A = [list(x) for x in zip(*(self.framesX))]
        A = torch.tensor(A, dtype=torch.float32)
        A = A.view(-1, A.shape[2])
        print("A:", A.shape)

        self.mn = torch.mean(A, dim=0)
        self.range = (torch.max(A, dim=0).values - torch.min(A, dim=0).values)/2.0
        self.range = torch.ones(self.range.shape, dtype=torch.float32)
        self.std = torch.std(A,dim=0)
        self.framesX = [(torch.tensor(item, dtype=torch.float32)-self.mn)/(self.std*self.range) for item in self.framesX]
        self.framesY = [(torch.tensor(item, dtype=torch.float32)-self.mn[:4])/(self.std[:4]*self.range[:4]) for item in self.framesY]

In [13]:
def getDataLoader(opt, csvPath) :
    """
    return torch.util.data.Dataloader for train, valid and test
    """
    # load dataset
    dataset = TrajectoryDataset()
    with open("Dataset.pickle", "wb") as output :
        pickle.dump(dataset, output)

    # split dataset into train test and valid 7:2:1
    numTrain = int(dataset.__len__()*0.7)
    numTest = int(dataset.__len__()*0.9) - numTrain
    numValid = int(dataset.__len__() - numTest - numTrain)
    train, valid, test = torch.utils.data.random_split(dataset, [numTrain, numValid, numTest])

    # create dataloader instance
    trainDataLoader = DataLoader(train, batch_size=opt["batchSize"], shuffle=True, drop_last=True)
    validDataLoader = DataLoader(valid, batch_size=opt["batchSize"], shuffle=False, drop_last=False)
    testDataLoader = DataLoader(test, batch_size=opt["batchSize"], shuffle=False, drop_last=False)

    return trainDataLoader, validDataLoader, testDataLoader, dataset

## **Trajectory LSTM Model 클래스 정의**

In [14]:
class TrajectoryLSTM(nn.Module) :
    def __init__(self, inputSize, outputSize, hiddenSize, numLayer, p) :
        # Inheritance
        super(TrajectoryLSTM, self).__init__()

        # Create LSTM Layer Instance
        self.lstm = nn.LSTM(hiddenSize, hiddenSize, num_layers=numLayer, bidirectional=False, batch_first=True, dropout=p)
        self.bilstm = nn.LSTM(hiddenSize, hiddenSize//2, num_layers=numLayer, bidirectional=True, batch_first=True, dropout=p)

        # Create FC Layer Instance
        self.input2lstm = nn.Linear(inputSize, hiddenSize)
        self.input2bilstm = nn.Linear(inputSize, hiddenSize)
        self.fc0 = nn.Linear(hiddenSize, 128)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, outputSize)
        self.input2output = nn.Linear(inputSize, 64)

        # Create Activation Layer Instance
        self.act = nn.Tanh()

    def forward(self, input) :
        lstmOutput, _ = self.lstm(self.input2lstm(input))
        bilstmOutput, _ = self.bilstm(self.input2bilstm(input))

        output = self.act(self.fc0(lstmOutput + bilstmOutput))
        output = self.act(self.fc1(output)) + self.input2output(input)
        output = self.fc2(output)

        return output

## **훈련 및 모델 하이퍼파라미터 선정**

In [15]:
opt = {"inputSize":30, "outputSize":4, "hiddenSize":256, "numLayer":5, "p":0.1,
       "batchSize":128, "numEpoch":100, "lr":1e-3, "seed":42}

## **Seed 고정**

In [16]:
import random
import numpy as np

In [17]:
def fixSeed(seed) :
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

## **훈련 과정 요약을 위한 Average Meter 인스턴스 생성**

In [18]:
class AverageMeter(object) :
  def __init__(self) :
    self.reset()

  def reset(self) :
    self.val = 0
    self.avg = 0
    self.sum = 0
    self.count = 0

  def update(self, val, n=1) :
    self.val = val
    self.sum += val*n
    self.count += n
    self.avg = self.sum / self.count

## **Trajectory LSTM 모델 훈련**

### **사용 Device 정하기 (GPU 또는 CPU)**

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(f"Device Type : {device}")

### **DataLoader 인스턴스 생성**

In [ ]:
trainDataLoader, validDataLoader, testDataLoader, dataset = getDataLoader(opt, "/content/WholeVdata2.csv")

### **Trajectory LSTM 모델 인스턴스 생성**

In [22]:
fixSeed(opt["seed"])

In [23]:
model = TrajectoryLSTM(opt["inputSize"], opt["outputSize"], opt["hiddenSize"], opt["numLayer"], opt["p"]).to(device)

### **Trajectory LSTM 모델 파라미터 개수 계산**

In [24]:
numParameter = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
print(f"Number of Trainable Parameters : {numParameter:,}")

### **손실 함수 인스턴스 생성**

In [26]:
criterion = nn.MSELoss()

### **Optimizer 인스턴스 생성**

In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=opt["lr"])

### **훈련 결과 저장을 위한 AverageMeter 인스턴스 생성**

In [28]:
trainLoss, validLoss = AverageMeter(), AverageMeter()

### **훈련 결과 저장을 위한 Python List 인스턴스 생성**

In [29]:
trainLossList, validLossList = [], []

In [30]:
bestValidLoss = torch.inf

### **훈련 진행**

In [ ]:
for epoch in range(1, opt["numEpoch"]+1) :
  ########################################################################################################################################

    trainBar = tqdm(trainDataLoader) # Create TQDM Instance
    trainLoss.reset() # Reset AverageMeter Instance

    model.train() # Train Mode

    for data in trainBar :
      input, target = data # Unpack Tuple Elements
      input, target = input.to(device), target.to(device) # Assign Device
      optimizer.zero_grad() # Set Gradient to 0
      pred = model(input) # Get Prediction
      loss = criterion(pred[:,-30:,2:4], target[:,-30:,2:4]) # Compute Loss
      loss.backward() # Back-Propagation
      optimizer.step() # Update Weight

      trainLoss.update(loss.detach().cpu().item(), opt["batchSize"]) # Compute Averaged Loss
      trainBar.set_description(desc=f"[{epoch}/{opt['numEpoch']}] [Train] < Loss:{trainLoss.avg:.4f} >")

    trainLossList.append(trainLoss.avg)

    ########################################################################################################################################

    validBar = tqdm(validDataLoader) # Create TQDM Instance
    validLoss.reset() # Reset AverageMeter Instance

    model.eval() # Evaulation Mode

    for data in validBar :
      input, target = data # Unpack Tuple Elements
      input, target = input.to(device), target.to(device) # Assign Device

      with torch.no_grad() :
        pred = model(input) # Get Prediction
        loss = criterion(pred[:,-30:,2:4], target[:,-30:,2:4]) # Compute Loss

        validLoss.update(loss.detach().cpu().item(), opt["batchSize"]) # Compute Averaged Loss
        validBar.set_description(desc=f"[{epoch}/{opt['numEpoch']}] [Valid] < Loss:{validLoss.avg:.4f} >")

    validLossList.append(validLoss.avg)

    if validLoss.avg < bestValidLoss :
      bestValidLoss = validLoss.avg
      torch.save(model.state_dict(), "Best-LSTM.pth")

    torch.save(model.state_dict(), "Latest-LSTM.pth")

In [ ]:
print(f"Best Valid Loss : {bestValidLoss:.4f}")

## **Trajectory LSTM 모델 훈련 과정 시각화**

In [ ]:
plt.plot(np.arange(opt["numEpoch"]), trainLossList, label="Train Loss")
plt.plot(np.arange(opt["numEpoch"]), validLossList, label="Valid Loss")
plt.legend(loc="best")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("[Trajectory] Train Loss vs. Valid Loss")
plt.show()

## **Trajectory LSTM 모델 추론**

### **Best Model 불러오기**

In [ ]:
weights = torch.load("/content/Best-LSTM.pth")
model.load_state_dict(weights, strict=True)

### **전처리에 사용한 통계값 불러오기**

In [ ]:
std = dataset.std[:4].to(device)
mn = dataset.mn[:4].to(device)
rg = dataset.range[:4].to(device)

### **Trajectory Model 추론 진행**

In [ ]:
predList, targetList = [], []

In [ ]:
testBar = tqdm(testDataLoader) # Create TQDM Instance

model.eval() # Evaulation Mode

for data in testBar :
    input, target = data # Unpack Tuple Elements
    input, target = input.to(device), target.to(device) # Assign Device

    with torch.no_grad() :
      pred = model(input)
      pred = (pred*(rg*std) + mn).detach().cpu().numpy()
      pred = scipy.signal.savgol_filter(pred, window_length=5, polyorder=2,axis=1)

      target = (target*(rg*std)+mn).detach().cpu().numpy()
      pred[:,:-30,:] = target[:,:-30,:]

      predList.append(pred)
      targetList.append(target)

### **추론 (예측) 결과 시각화**

In [ ]:
plt.figure(figsize=(10, 5))

index = 0
plt.plot(predList[0][index,:,2], predList[0][index,:,3], "r", label="Prediction")
plt.plot(targetList[0][index,:,2], targetList[0][index,:,3], "g", label="Ground-Truth")
plt.xlabel("Local X Coordinate")
plt.ylabel("Local Y Coordinate")
plt.title("Trajectory Prediction")
plt.legend(loc="best")
plt.show()

## **Trajectory LSTM 모델 구조를 바꾸어 가면서 성능을 올려보세요**

In [ ]:
# Option Dictionary 입력

### **Trajectory LSTM Model 클래스 정의**

In [ ]:
# 모델 구조 설계

### **Trajectory LSTM 모델 훈련 (MSE)**

#### **LSTM 모델 인스턴스 생성**

In [ ]:
# 시드 고정

In [ ]:
# 모델 인스턴스 생성

#### **LSTM 모델 파라미터 개수 계산**

In [ ]:
# 모델 파라미터 계산

In [ ]:
# 모델 파라미터 개수 출력

#### **손실 함수 인스턴스 생성**

In [ ]:
# MSE 손실 함수 인스턴스 생성

#### **Optimizer 인스턴스 생성**

In [ ]:
# Adam Optimizer 인스턴스 생성

#### **훈련 진행**

In [ ]:
# 모델 훈련 코드 작성

### **Trajectory LSTM 모델 훈련 과정 시각화**

In [ ]:
# 훈련 과정 시각화 코드 작성

### **모델 성능 평가**

#### **Best Model 불러오기**

In [ ]:
# Best Model 불러오기

#### **전처리에 사용한 통계값 불러오기**

In [ ]:
# 통계값 계산

#### **Trajectory Model 추론 진행**

In [ ]:
# List 인스턴스 생성

In [ ]:
# 모델 추론 진행

#### **추론 (예측) 결과 시각화**

In [ ]:
# 예측 결과 시각화

### **Trajectory LSTM 모델 훈련 (MAE)**

#### **LSTM 모델 인스턴스 생성**

In [ ]:
# 시드 고정

In [ ]:
# 모델 인스턴스 생성

#### **LSTM 모델 파라미터 개수 계산**

In [ ]:
# 모델 파라미터 계산

In [ ]:
# 모델 파라미터 개수 출력

#### **손실 함수 인스턴스 생성**

In [ ]:
# MAE 손실 함수 인스턴스 생성 -> nn.L1Loss() 사용

#### **Optimizer 인스턴스 생성**

In [ ]:
# Adam Optimizer 인스턴스 생성

#### **훈련 진행**

In [ ]:
# 모델 훈련 코드 작성

### **Trajectory LSTM 모델 훈련 과정 시각화**

In [ ]:
# 훈련 과정 시각화 코드 작성

### **모델 성능 평가**

#### **Best Model 불러오기**

In [ ]:
# Best Model 불러오기

#### **전처리에 사용한 통계값 불러오기**

In [ ]:
# 통계값 계산

#### **Trajectory Model 추론 진행**

In [ ]:
# List 인스턴스 생성

In [ ]:
# 모델 추론 진행

#### **추론 (예측) 결과 시각화**

In [ ]:
# 예측 결과 시각화